# Inverse problem of the reconstruction of the Neumann BCs for Laplace's equation

__Formulation of Inverse Problem__

__Direct Problem__
- Equation: $A * x = f$
- $A$: Matrix of Direct Problem
  - shape: ($\text{dof}$, $\text{dof}$)
- $x$: Solution of Direct Problem
  - shape: ($\text{dof}$)
- $f$: Load Vector of Direct Problem
  - shape: ($\text{dof}$)
- $F$: Projection Matrix from Normal Derivative in given points (Mass Matrix over Boundary Elements)
  - shape: ($\text{dof}$, $g$)

__Inverse Problem__
- Equation: $M * y = \text{grad}$
- $M$: Resulting Matrix of Inverse Problem
  - shape: ($3p$, $\text{dof}$)
- $y$: Normal Derivative in Boundary Points
- $\text{grad}$: Concatenation of Grad Vectors ($\text{grad}_x, \text{grad}_y, \text{grad}_z$)

__Additional Variables__
- $P_x, P_y, P_z$: Projection Matrices from Solution to Partial Derivative
  - shape: ($p$, $\text{dof}$)
- $M_x = P_x \times A^{-1} \times F$: Need batching with Direct Solvers (spsolve or better factorized to reduce amount of time)
  - Note: $A$ is singular, so we need to be careful with solving the system
- $M = (M_x, M_y, M_z)$: Concatenation of Inverse Problem Matrices
- $\text{dof}$: Degrees of Freedom of Direct Problem
- $p$: Number of Measuring Points
- $g$: Number of Boundary Points with Known Normal Derivative (Is it all boundary points?)

__About calculating M__

Computing $M$ consists of solving a lot of systems of linear equations:

$$A^{-1} * F = X$$

is same with solving SLAE with matrix rhs:

$$AX = F$$

Result of this multipication can be a big dense (90% density) matrix so we should solve SLAE with one or several columns form $F$.

Probable algorithm for getting $M$ (Under investigation, because it gives big error (e.g. `1e-2` instead of `1e-11`))

```python
for f in F:

  x = spsolve(A, f) # f.shape: (dof, 1)
  
  Mx[i] = P_x @ x
```

__Questions__

1. How to deal with $A$ is singular? It affects calculation of $M$.

2. How to assemble $F$, if we have rhs (source) in direct problem? (If equation is not Laplace's)

## Preprocessing

Import the required libraries

In [1]:
import numpy as np
import scipy as sp
import cupy as cp
import cupy.sparse.linalg as cpsl

from tqdm import tqdm

import pyquasar as pq

np.set_printoptions(precision=3, suppress=False)
sp.sparse.linalg.use_solver(useUmfpack=False)

Load the `.geo` file and generate the mesh for the problem using GMSH.

`refine_k` is the number of times the mesh is refined.

In [2]:
mesh = pq.Mesh.load("circle_splitted.msh", basis_order=2, refine_k=1)
mesh

<Mesh object summary 
	Numeration: global
	Domains: [<MeshDomain object summary
	Material: air
	Total elements number: 86016
	Element type: Tetrahedron 10 NC; Count: 86016
	Boundary type: neumann; Tag: 4; Element type: Triangle 6 NC; Count: 8192.
	Boundary type: neumann; Tag: 5; Element type: Triangle 6 NC; Count: 896.
	Boundary type: neumann; Tag: 6; Element type: Triangle 6 NC; Count: 896.
>]>

In [3]:
mesh.domains[0].vertices[..., 2] = mesh.domains[0].vertices[..., 2] + 1.6
mesh.domains[0].vertices[..., 2].min()

0.0

## Direct Problem

Assemble & solve the direct problem to obtain problem matrix, load vector and the solution `u`.

Note that mesh consists of only Neumann boundary conditions and therefore the solution `u` is need to be orthogonalized.

In [4]:
# Define the materials dictionary
materials = {
  "air": {"air": 0},
}

# Create a list of FemDomain objects from the mesh domains
domains = [pq.FemDomain(domain) for domain in mesh.domains]

# Create a FemProblem object with the domains
problem = pq.FemProblem(domains)

# Assemble the problem using the materials dictionary
problem.assembly(materials, batch_size=1024)

problem._matrix = problem._matrix.tocsr()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix = problem._matrix.tocsc()
problem.matrix.data[problem.matrix.indptr[-2] : problem.matrix.indptr[-1]] = 0
problem._matrix[-1, -1] = 1
problem._matrix.eliminate_zeros()

# Print the degree of freedom count
print(f"DOF: {problem.dof_count}")

DOF: 124865


## Inverse Problem

Get points of interest

In [5]:
data = np.load("data/data.npz")

xs = data["xs"]
ys = data["ys"]
zs = data["zs"]

grad_x = data["grad_x"]
grad_y = data["grad_y"]
grad_z = data["grad_z"]

data.close()

grad = np.concatenate([grad_x[:7200], grad_y[:7200], grad_z[:7200]])
grad_cp = cp.array(grad)

pts = np.concatenate([xs[:7200, None], ys[:7200, None], zs[:7200, None]], axis=1)
pts.shape

(7200, 3)

Project the solution gradient into the points of interest

In [6]:
proj_grad = problem.project_grad_into(cp.asarray(pts), batch_size=128)

100%|██████████| 57/57 [00:02<00:00, 22.00it/s]


Get the Mass matrix over the Neumann type boundary

In [7]:
F = problem.mass_boundary(["neumann"])
F_cp = cp.sparse.csc_matrix(F)

Initialize the inverse problem matrix

In [8]:
m_shape = pts.shape[0], F.shape[1]
Mx_cp = cp.zeros(m_shape)
My_cp = cp.zeros(m_shape)
Mz_cp = cp.zeros(m_shape)

Get factorization of direct problem matrix

In [9]:
lu_cp = cp.sparse.linalg.splu(cp.sparse.csc_matrix(problem.matrix))

Iteratively calculate the inverse problem matrix

In [10]:
proj_grad_cp = (
  cp.sparse.csr_matrix(proj_grad[0]),
  cp.sparse.csr_matrix(proj_grad[1]),
  cp.sparse.csr_matrix(proj_grad[2]),
)

In [11]:
batch_size = 256
num_batches = F_cp.shape[1] // batch_size

if F_cp.shape[1] % batch_size != 0:
  num_batches += 1

for i in tqdm(range(num_batches)):
  sol = lu_cp.solve(F_cp.T[i * batch_size : (i + 1) * batch_size].T.toarray())
  
  Mx_cp[:, i * batch_size : (i + 1) * batch_size] = (proj_grad_cp[0] @ sol)
  My_cp[:, i * batch_size : (i + 1) * batch_size] = (proj_grad_cp[1] @ sol)
  Mz_cp[:, i * batch_size : (i + 1) * batch_size] = (proj_grad_cp[2] @ sol)
M_cp = cp.concatenate([Mx_cp, My_cp, Mz_cp])

del Mx_cp
del My_cp
del Mz_cp
del F_cp
del proj_grad_cp

mempool = cp.cuda.MemoryAsyncPool()
mempool.free_all_blocks()

  0%|          | 0/79 [00:00<?, ?it/s]

100%|██████████| 79/79 [01:18<00:00,  1.01it/s]
/tmp/ipykernel_624284/2234959353.py:21: FutureWarning: cupy.cuda.MemoryAsyncPool is experimental. The interface can change in the future.
  mempool = cp.cuda.MemoryAsyncPool()


Iteratively calculate the inverse problem solution

In [20]:
res_flow_cp, istop, itn, normr, normar = cpsl.lsmr(M_cp, grad_cp, atol=1e-15, btol=1e-15, maxiter=100)[:5]

rerr = cp.linalg.norm(M_cp @ res_flow_cp - grad_cp) / np.linalg.norm(grad_cp)

print(f"The reason of stopping: {istop}")
print(f"Number of iterations: {itn}")
print(f"Norm of the residual: {normr:.2e}")
print(f"Norm of modified residual: {normar:.2e}")
print(f"Relative error of inverse problem solution (M @ y = grad): {rerr:.2e}")

The reason of stopping: 7
Number of iterations: 100
Norm of the residual: 1.76e-01
Norm of modified residual: 8.41e-04
Relative error of inverse problem solution (M @ y = grad): 9.49e-04


In [21]:
# M_pinv_cp = cp.linalg.pinv(M_cp, rcond=1e-11)
# res_flow_cp = M_pinv_cp @ grad_cp

# normr_pinv = cp.linalg.norm(M_cp @ res_flow_cp - grad_cp) / cp.linalg.norm(grad_cp)
# normar_pinv = cp.linalg.norm(M_cp.T @ M_cp @ res_flow_cp - M_cp.T @ grad_cp) / cp.linalg.norm(M_cp.T @ grad_cp)
# print(f"Norm of modified residual: {normar_pinv:.2e}")
# print(f"Norm of residual (M @ y = grad): {normr_pinv:.2e}")

In [22]:
res_flow = cp.asnumpy(res_flow_cp)

Test through direct problem

In [23]:
# Get the kernel for Gram-Schmidt orthogonalization
kernel = problem.domains[0].kernel

# Calculate load vector via inverse problem result flow
problem._load_vector = np.zeros_like(problem._load_vector)
problem._load_vector += F @ res_flow

# Solve the problem
test_sol = problem.solve(atol=1e-15)

# Perform the Gram-Schmidt orthogonalization
test_sol -= kernel[0] * (test_sol @ kernel[0]) / (kernel[0] @ kernel[0])

In [24]:
test_pts = np.concatenate([xs[7200:, None], ys[7200:, None], zs[7200:, None]], axis=1)
test_true_grad = np.concatenate([grad_x[7200:], grad_y[7200:], grad_z[7200:]])
test_proj_grad = problem.project_grad_into(cp.asarray(test_pts), batch_size=512)

100%|██████████| 287/287 [00:19<00:00, 14.67it/s]


In [25]:
test_grad_x = test_proj_grad[0] @ test_sol
test_grad_y = test_proj_grad[1] @ test_sol
test_grad_z = test_proj_grad[2] @ test_sol

test_grad = np.concatenate([test_grad_x, test_grad_y, test_grad_z])

In [26]:
diff_x = test_grad_x - grad_x[7200:]
diff_y = test_grad_y - grad_y[7200:]
diff_z = test_grad_z - grad_z[7200:]

In [27]:
err = np.sqrt((diff_x**2 + diff_y**2 + diff_z**2) / (diff_x.shape[0]))
print(f"Result error: {err.max():.2e}")

Result error: 2.34e-04
